In [1]:
import torch
import numpy as np
from torchmetrics.retrieval import RetrievalMAP

In [2]:
K=10

In [3]:
truth = torch.rand(2, 20)
truth = torch.where(truth >= 0.5, 1, 0)
preds = torch.rand(2, 20)

print(preds)
print(truth)

tensor([[0.4082, 0.0279, 0.5182, 0.9956, 0.5743, 0.2198, 0.1459, 0.0024, 0.4262,
         0.2257, 0.3194, 0.4648, 0.7320, 0.1438, 0.1873, 0.6121, 0.8604, 0.1967,
         0.6816, 0.1389],
        [0.5069, 0.6264, 0.5795, 0.4414, 0.2198, 0.2324, 0.7687, 0.7223, 0.8915,
         0.3777, 0.2694, 0.6389, 0.0620, 0.2020, 0.6193, 0.0183, 0.3427, 0.0127,
         0.9527, 0.3974]])
tensor([[0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
        [1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1]])


In [4]:
def get_pre_at_k(preds, truth, k):
    top_k_indices = torch.topk(preds, k=k)[1]
    truth_on_topk = torch.gather(truth, 1, top_k_indices)
    pre_k = truth_on_topk.sum(dim=1) / k
    return pre_k

In [5]:
top_k_indices = torch.topk(preds, k=K)[1]
top_k_indices

tensor([[ 3, 16, 12, 18, 15,  4,  2, 11,  8,  0],
        [18,  8,  6,  7, 11,  1, 14,  2,  0,  3]])

In [6]:
truth_on_topk = torch.gather(truth, 1, top_k_indices)
truth_on_topk

tensor([[1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
        [1, 0, 1, 0, 1, 0, 0, 0, 1, 1]])

In [7]:
pre_at_k = []

for i in range(truth_on_topk.size(1)):
    curr_pre_at_k = get_pre_at_k(preds, truth, i+1)
    pre_at_k.append(curr_pre_at_k.view((2, 1)))
#     print(i, curr_pre_at_k)
# print(pre_at_k)
pre_at_k = torch.cat(pre_at_k, dim=1)
pre_at_k

tensor([[1.0000, 0.5000, 0.6667, 0.7500, 0.8000, 0.8333, 0.7143, 0.7500, 0.7778,
         0.7000],
        [1.0000, 0.5000, 0.6667, 0.5000, 0.6000, 0.5000, 0.4286, 0.3750, 0.4444,
         0.5000]])

In [8]:
ap = (truth_on_topk * pre_at_k).sum(dim=1) / truth_on_topk.sum(dim=1)
ap

tensor([0.7968, 0.6422])

In [9]:
map = ap.mean()
map

tensor(0.7195)

# Verification

In [10]:
map = RetrievalMAP(top_k=K)

indexes = torch.tensor([[i] * truth.size(1) for i in range(truth.size(0))])
map(preds.flatten(), truth.flatten(), indexes=indexes.flatten())

tensor(0.7195)